In [1]:
!pip install -q crewai crewai_tools langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.7/366.7 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 63.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 46.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 657.0/657.0 kB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 62.4 MB/s eta 

In [11]:
import warnings
warnings.filterwarnings('ignore')
import os
os.environ["OPENAI_API_KEY"] = ""
os.environ["OPENAI_MODEL_NAME"] = "gpt-4o-mini"
os.environ["SERPER_API_KEY"] = "9449a9f30f26c876d3db22f10300c548954455ca"

In [ ]:
# User input
# - Location
# - Duration
# - Budget
# - Startdate
# - num of people

# Travel Plan
# Sightseeing - top attractions
# fooding - food options / suggestion
# hotel - where  which hotels
# family friendly / kids friendly place
# ITINERY - Day by Day

In [3]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

In [4]:
from crewai import Agent, Task, Crew

In [15]:
# From tools import results of search trip and scrap page

def get_travel_researcher():
  return Agent(
      role = "Travel Researcher",
      goal = "Find best attractions, accomodations, travel tips",
      backstory = "Expert in finding travel destinations , must visit spots",
      tools = [scrape_tool,search_tool],
      verbose = True
  )

def get_travel_planner():
  return Agent(
      role = "Travel Planner",
      goal = "Create a day by day travel itinery",
      backstory = "Expert in creating enjoyable , family friendy amazing travel plans",
      verbose = True
  )

def get_budget_Analyst():
   return Agent(
      role = "Budget Controller",
      goal = "Check if itinery fits the budget and suggest imporvements. Get the costs for the hotels on the specific dates mentioned and latest costs from latest websites",
      backstory = "Expert in cost estimation and travel budget optimization ensuring travel plans stay within budget",
      verbose = True
  )

# User input
user_input ={
    "location": "Singapore",
    "duration": 5,
    "budget" : 200000,
    "start_date":"2025-10-01",
    "num_people":4
}


# Agents
researcher = get_travel_researcher()
planner = get_travel_planner()
budget_analyst = get_budget_Analyst()

# Tasks
research_task = Task(
    description = f""" Use search and scraping tools to find
    - top attractions
    - family/ kid friendly places
    - best hotels / best hotel areas
    - travel tips
    - local food suggestions for {user_input['location']}
    """,
    expected_output = "A bullet point summary of attractions, hotels or hotel areas and travel tips along with food suggestions or food places",
    agent = researcher
)

planning_task = Task(
    description = f"""Create a {user_input['duration']} day itinery for {user_input['location']} starting from {user_input['start_date']} for {user_input['num_people']} people
    using the research findings. Include specifically sightseeing, rest time , hotel stay , food option details.
    """,
    expected_output = "A day-wise itinery with optional notes for each sighseeing or activity proposed",
    agent = planner
)

budget_optimization_task = Task(
    description = f"""Estimate the total travel cost in INR based on the itinery propsed. Allowed budget is INR{user_input['budget']} for {user_input['num_people']} people.
    Suggest cost cutting alternatives to ensure the itinery stays within budget if the proposed itinery exceeds budget
    """,
    expected_output = "Total cost estimate and suggestions to reduce cost to fit budget of needed",
    agent = budget_analyst
    # we can introduce tools for accurate calculation . Right now LLM is being used for calculation - result may or may not be exact
)


#Crew
crew = Crew(
    agents = [researcher,planner,budget_analyst],
    tasks = [research_task,planning_task,budget_optimization_task],
    verbose = True
)

In [16]:
result = crew.kickoff()

╭─────────────────────────── Crew Execution Started ───────────────────────────╮
│                                                                              │
│  Crew Execution Started                                                      │
│  Name: crew                                                                  │
│  ID: 48d51b09-c4e4-4439-89da-abca2c044117                                    │
│  Tool Args:                                                                  │
│                                                                              │
│                                                                              │
╰──────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 861c88d2-c61d-45e1-96cc-92b0d2f91d05
    Status: Executing Task...╭────────────────────────────── 🤖 Agent Started ──────────────────────────────╮
│                                                                              │
│  Agent: Travel R

In [17]:
from IPython.display import display, Markdown, HTML
Markdown(result.raw)

### Total Travel Cost Estimate for 4 People in INR

1. **Accommodation Costs:**
   - **Marina Bay Sands** (5 days): Approx. INR 40,000 per night
   - Total for 5 nights: 5 * 40,000 = **INR 200,000**

2. **Food Costs:**
   - Breakfast at hotel: (included in hotel rate)
   - Lunch (5 days average INR 1,500 per meal): 5 * 1,500 = **INR 7,500**
   - Dinner (5 days average INR 2,500 per meal): 5 * 2,500 = **INR 12,500**
   - Total for meals: **INR 20,000**

3. **Attraction Costs:**
   - **Gardens by the Bay:** INR 1,000 (entry for 4)
   - **Universal Studios Singapore:** Approx. INR 12,000 (INR 3,000 each for 4)
   - **Singapore Zoo:** Approx. INR 4,000 (INR 1,000 each for 4)
   - **S.E.A. Aquarium:** Approx. INR 4,000 (INR 1,000 each for 4)
   - **Adventure Cove Waterpark:** Approx. INR 4,000 (INR 1,000 each for 4)
   - Total for attractions: **INR 25,000**

4. **Transport Costs:**
   - MRT passes (average INR 200 per day): 5 days * 4 people * 200 = **INR 4,000**

Adding all these together:
- **Accommodation:** INR 200,000
- **Food:** INR 20,000
- **Attractions:** INR 25,000
- **Transport:** INR 4,000

**Total Estimated Cost: INR 249,000**

### Budget and Improvements:
The proposed itinerary exceeds the budget of **INR 200,000** by **INR 49,000**. Here are the cost-cutting alternatives to fit within the budget:

1. **Accommodation:** 
   - Instead of Marina Bay Sands, consider staying in **Chinatown** or **Clarke Quay** where hotels can average around INR 15,000 to INR 20,000 per night, potentially reducing the total hotel cost to around **INR 75,000** for 5 nights.

2. **Food:**
   - Opt for hawker centers for all meals instead of some restaurant dinners. This could lower the meal budget greatly. Averages could go down to **INR 10,000** total for the 5 days if meals average closer to **INR 500 per meal**.

3. **Attractions:**
   - Consider choosing 3 key attractions instead of all listed or use a combo ticket for some of the more child-friendly places which may reduce entry fees, potentially lowering this category to around **INR 15,000**.

4. **Transport:**
   - Reduce transport costs by leveraging more walks, especially in areas like Marina Bay, which are pedestrian-friendly. This could cut down transport costs by half, leaving roughly **INR 2,000**.

---

### Implementing Improvements

- **New Accommodation Costs:** INR 75,000
- **New Food Costs:** INR 10,000
- **New Attraction Costs:** INR 15,000
- **New Transport Costs:** INR 2,000

**Adjusted Total Cost: INR 102,000**

By selecting budget-friendly hotels, focusing on local food, and carefully choosing attractions, the total could reduce to around **INR 102,000**, keeping it comfortably under your INR 200,000 budget while still ensuring a rich travel experience. 

Happy planning for your trip to Singapore! Enjoy your adventure!

In [ ]:
# Ambiguous Ask - Iterations within ourselves ...( Autogen)
# I need to do an RCA of why sales went down?
# HYpothesis - a) Product category b) Region c) Product x Region d) App issue